In [7]:
import joblib
import numpy as np
import pandas as pd

In [8]:
data = pd.read_csv('../data_sample/titanic_test.csv')
X_test = data.drop(columns='Survived')
y_test = data['Survived']

modelo = joblib.load('../model/modelo_pipeline.joblib')

In [9]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [10]:
modelo

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('Preprocesado',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('Process_Numeric_Log',
                                                                         Pipeline(steps=[('Impute_Mean',
                                                                                          SimpleImputer()),
                                                                                         ('Apply_Log',
                                                                                          FunctionTransformer(feature_names_out='one-to-one',
                                                                                                              func=<ufunc 'log1p'>)),
                                                                                         ('SScaler',
                                                                                          StandardScaler())]),
                                                                         ['Fare']),
                                                                        ('Process_Numeric',
                                                                         Pipeline(steps=[(...
                                                      max_delta_step=None,
                                                      max_depth=None,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=None,
                                                      num_parallel_tree=None, ...))]),
             n_jobs=-1,
             param_grid={'Modelo__learning_rate': [0.1, 0.2, 0.5, 1.0],
                         'Modelo__max_depth': [1, 2, 4, 8],
                         'Modelo__n_estimators': [10, 100, 200, 400]},
             scoring='accuracy', verbose=1)

In [11]:
X_test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [12]:
def X_data(age:float, sex:str, pclass:int, fare:float):
    '''
    age = edad del pasajero
    sex  = 'female' | 'male'
    fare = tarifa del billete
    clase = 3
    '''

In [13]:
def validate_age(age):
    if age is None or not (0 <= age <= 100):
        return False, "Edad fuera de rango (0-100)"
    return True, age

def validate_sex(sex):
    if sex not in ['male', 'female']:
        return False, "Sexo inválido, debe ser 'male' o 'female'"
    return True, sex

def validate_pclass(pclass):
    if pclass not in [1, 2, 3]:
        return False, "Clase inválida, debe ser 1, 2 o 3"
    return True, pclass

def validate_fare(fare):
    if fare is None or fare < 0:
        return False, "Tarifa inválida: debe ser >= 0"
    return True, fare

def X_data(age: float, sex: str, pclass: int, fare: float, column_order=None):
    X = pd.DataFrame([{
        'Age': age,
        'Sex': sex,
        'Pclass': pclass,
        'Fare': fare
    }])
    if column_order is not None:
        X = X.reindex(columns=column_order, fill_value=0)
    return X

In [14]:
def predict_survival(model, age, sex, pclass, fare, column_order=None):
    # Validaciones
    valid_age, age_result = validate_age(age)
    valid_sex, sex_result = validate_sex(sex)
    valid_pclass, pclass_result = validate_pclass(pclass)
    valid_fare, fare_result = validate_fare(fare)

    if not (valid_age and valid_sex and valid_pclass and valid_fare):
        return (
            f"Error en datos: "
            f"{age_result if not valid_age else ''} "
            f"{sex_result if not valid_sex else ''} "
            f"{pclass_result if not valid_pclass else ''} "
            f"{fare_result if not valid_fare else ''}"
        )
    
    X = X_data(age_result, sex_result, pclass_result, fare_result, column_order)
    prediction = model.predict(X)
    return prediction[0]

In [16]:
age = 20
sex  = 'female'
fare = 1
clase = 0

X = X_data(age, sex, clase, fare)
modelo.predict(X)

array([1])